In [1]:
#import libraries.
import numpy as np
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
from tensorflow.python.keras.utils.data_utils import Sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import keras
from keras import regularizers
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import sounddevice as sd
import tensorflow as tf
from mir_eval import separation 
from pystoi.stoi import stoi 
import h5py
from keras.callbacks import LearningRateScheduler
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation
import math
from sklearn.utils import shuffle
from sklearn import preprocessing
import soundfile as sf
import matplotlib.pyplot as plt
from pypesq import pesq
import soundfile as sf

Using TensorFlow backend.


In [2]:
#import libraries.
# import matplotlib.pyplot as plt
# from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
import time
import pickle
from keras import backend as K
from tensorflow.keras.models import load_model
# from keras.models import load_model

In [3]:
def reconstruct(wave,angle):
    recon1 = wave*np.cos(angle)+wave*np.sin(angle)*1j
#     recon = np.sqrt(np.power(10, wave))
#     recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=128, win_length=512, window='hann')
    return recon

# Loading Model

In [4]:
estimator_A = load_model('Models/Two_stage/512_3/saved-model-adam-046-0.0528.hdf5')
name0 = '512_3_46'

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [5]:
data_series = str(0)

In [6]:
h5f = h5py.File('TSP/Organized/concatenated/Test_TSP_mixed_FFT'  + data_series + '.hdf5','r')
data_shape = h5f['Test_TSP_mixed_FFT' + data_series].shape
print(data_shape)
h5f.close()

(257, 29014)


In [7]:
h5f = h5py.File('TSP/Organized/concatenated/Test_all_len'  + data_series + '.hdf5','r')
all_len = h5f['Test_all_len' + data_series][:]

# Data 

In [8]:
# all
test_point_start0 = 0
test_point_stop0 = data_shape[1]

In [9]:
# FM
FM_test_point_start0 =  0
FM_test_point_stop0 = all_len[36]

In [10]:
# FF
FF_test_point_start0 =  all_len[36]+1
FF_test_point_stop0 = all_len[72]

In [11]:
# MM
MM_test_point_start0 =  all_len[72] + 1
MM_test_point_stop0 = data_shape[1]

In [12]:
indx = [test_point_start0, test_point_stop0, FM_test_point_start0, FM_test_point_stop0, FF_test_point_start0, 
        FF_test_point_stop0, MM_test_point_start0, MM_test_point_stop0]

In [13]:
fft_samples = 257

# saving voices

In [14]:
k = 0
SR = 16000
for i in range(0,len(indx),2):
    test_point_start0 = indx[i]
    test_point_stop0 = indx[i+1]
    h5f = h5py.File('TSP/Organized/concatenated/Test_TSP_mixed_FFT'  + data_series + '.hdf5','r')
    test_input = h5f['Test_TSP_mixed_FFT' + data_series][:, test_point_start0 : test_point_stop0]
    h5f = h5py.File('TSP/Organized/concatenated/Test_TSP_clean_FFTs'  + data_series + '.hdf5','r')
    test_target = h5f['Test_TSP_clean_FFTs' + data_series][:, test_point_start0 : test_point_stop0]
    h5f = h5py.File('TSP/Organized/concatenated/Test_TSP_mixed_phase'  + data_series + '.hdf5','r')
    mixed_phase = h5f['Test_TSP_mixed_phase' + data_series][:, test_point_start0 : test_point_stop0]
    h5f = h5py.File('TSP/Organized/concatenated/Test_TSP_clean_phases'  + data_series + '.hdf5','r')
    clean_phase = h5f['Test_TSP_clean_phases' + data_series][:, test_point_start0 : test_point_stop0]
    norms = np.zeros(test_input.shape[1])
    for j in range(test_input.shape[1]):
        norms[j] = np.linalg.norm(test_input[:,j])
    test_input = preprocessing.normalize(test_input, norm='l2', axis=0, copy=True)
    prediction = estimator_A.predict(test_input.T)
    test_input = np.multiply(test_input,norms)
    prediction0 = np.multiply(test_input.T, prediction[:,0:fft_samples])
    prediction1 = np.multiply(test_input.T, prediction[:,fft_samples:])
    speaker1 = reconstruct(prediction0, mixed_phase.T) + 2.179366134322788e-06
    speaker2 = reconstruct(prediction1, mixed_phase.T) + 2.179366134322788e-06
    clean_recon1 = reconstruct(test_target[0:fft_samples,:].T, clean_phase[0:fft_samples,:].T) + 2.179366134322788e-06
    clean_recon2 = reconstruct(test_target[fft_samples:,:].T, clean_phase[fft_samples:,:].T) + 2.179366134322788e-06
    mixed_recon = reconstruct(test_input.T, mixed_phase.T) + 2.179366134322788e-06
    if k == 0:
        sf.write('TSP/Predicted/Voices/all_speaker1_' + name0 + '.wav', speaker1, SR)
        sf.write('TSP/Predicted/Voices/all_speaker2_' + name0 + '.wav', speaker2, SR)
        sf.write('TSP/Predicted/Voices/all_clean1_' + name0 + '.wav', clean_recon1, SR)
        sf.write('TSP/Predicted/Voices/all_clean2_' + name0 + '.wav', clean_recon2, SR)
        sf.write('TSP/Predicted/Voices/all_mixed_' + name0 + '.wav', mixed_recon, SR)
    if k == 1:
        sf.write('TSP/Predicted/Voices/FM_speaker1_' + name0 + '.wav', speaker1, SR)
        sf.write('TSP/Predicted/Voices/FM_speaker2_' + name0 + '.wav', speaker2, SR)
        sf.write('TSP/Predicted/Voices/FM_clean1_' + name0 + '.wav', clean_recon1, SR)
        sf.write('TSP/Predicted/Voices/FM_clean2_' + name0 + '.wav', clean_recon2, SR)
        sf.write('TSP/Predicted/Voices/FM_mixed_' + name0 + '.wav', mixed_recon, SR)
    if k == 2:
        sf.write('TSP/Predicted/Voices/FF_speaker1_' + name0 + '.wav', speaker1, SR)
        sf.write('TSP/Predicted/Voices/FF_speaker2_' + name0 + '.wav', speaker2, SR)
        sf.write('TSP/Predicted/Voices/FF_clean1_' + name0 + '.wav', clean_recon1, SR)
        sf.write('TSP/Predicted/Voices/FF_clean2_' + name0 + '.wav', clean_recon2, SR)
        sf.write('TSP/Predicted/Voices/FF_mixed_' + name0 + '.wav', mixed_recon, SR)
    if k == 3:
        sf.write('TSP/Predicted/Voices/MM_speaker1_' + name0 + '.wav', speaker1, SR)
        sf.write('TSP/Predicted/Voices/MM_speaker2_' + name0 + '.wav', speaker2, SR)
        sf.write('TSP/Predicted/Voices/MM_clean1_' + name0 + '.wav', clean_recon1, SR)
        sf.write('TSP/Predicted/Voices/MM_clean2_' + name0 + '.wav', clean_recon2, SR)
        sf.write('TSP/Predicted/Voices/MM_mixed_' + name0 + '.wav', mixed_recon, SR)
    k = k + 1